In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%capture
! pip install datasets

## 1. Fine-Tuning 준비

In [ ]:
from datasets import load_dataset

klue_mrc_train = load_dataset('klue', 'mrc', split='train')
klue_mrc_test = load_dataset('klue', 'mrc', split='validation')

df_train = klue_mrc_train.to_pandas()
df_test = klue_mrc_test.to_pandas()

df_train.sample(1)

README.md:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

,title,context,news_category,source,guid,is_impossible,question_type,question,answers
12266,결정 못하는 것이 결점이라면 …,요즘 ‘결정 장애 세대’라는 말이 유행이다. 스스로 결정하지 못하고 “글쎄요” “원...,책마을,hankyung,klue-mrc-v1_train_10843,False,1,사람들이 스스로 결정내리는 걸 어려워하는 것을 칭하는 말은 무엇인가?,"{'answer_start': [4], 'text': ['결정 장애']}"


In [ ]:
df_train = df_train[['title', 'question', 'context']]
df_test = df_test[['title', 'question', 'context']]

df_train.shape, df_test.shape

((17554, 3), (5841, 3))

In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer('whatwant/klue-roberta-base-klue-sts')
sentence_model

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/14.8k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17554 entries, 0 to 17553
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     17554 non-null  object
 1   question  17554 non-null  object
 2   context   17554 non-null  object
dtypes: object(3)
memory usage: 411.6+ KB


In [ ]:
df_train.sample(2)

,title,question,context
13202,황개,누가 적벽대전에서 승리했는가?,처음에는 영릉군의 군리로 있었다. 손견이 의병을 일으키자 이를 따라갔다. 손견이 죽...
4023,"제17회 EBS국제다큐영화제(EIDF2020), ‘페스티벌 초이스’ 심사위원 8인 공개",이승준 감독이 심사를 맡은 영화제의 시작은 언제인가?,제17회 EBS국제다큐영화제(이하 EIDF2020)가 오는 8월 17일 개막을 앞두...


In [ ]:
title = '자비가 일족'

temp1 = df_train.query(f'title != "{title}"')
temp1.sample(2)

,title,question,context
4108,"쌍용건설, 3년 만에 재건축 수주",쌍용건설이 등촌1주택재건축구역 공사 직전에 재건축을 맡은 것은 몇 년도인가?,쌍용건설이 3년 만에 재건축 수주에 성공했다.쌍용건설은 서울 강서구 등촌동 등촌1주...
11736,김용석 (법조인),김용석이 가사 항고 소송 연구회 회장을 맡아 있을 때 주장했던 제도는?,1963년 서울시에서 태어난 김용석은 휘문고등학교와 서울대학교 법학과를 졸업하고 1...


In [ ]:
temp2 = temp1.sample(1)['context']
temp2

,context
4284,“글로벌 인수합병(M&A)이야말로 ‘일대일로’(一帶一路·육해상을 잇는 현대판 실크로...


In [ ]:
temp2.values

array(['“글로벌 인수합병(M&A)이야말로 ‘일대일로’(一帶一路·육해상을 잇는 현대판 실크로드) 전략을 실현할 강력한 수단입니다.”(왕친민 중화전국공상업연합회 회장)“중국 경제가 급성장하면서 기업 간 경쟁도 격화하고 있습니다. 주변국 시장에 대한 적극적인 M&A를 통해 효율적인 성장을 추구해야 합니다.”(류밍캉 전 중국은행감독위원회 위원장)지난 5일 중국 상하이에서 중화전국공상업연합회 주최로 열린 아시아M&A협회 연차포럼장을 찾은 중국 정부 및 경제단체 관계자들은 “글로벌 M&A가 중국의 새 성장동력이 될 것”이라고 입을 모았다. 아시아M&A협회는 아시아 국가 간 M&A 관련 교류·협력을 위해 설립된 협회로 한국M&A투자협회를 비롯해 중국 호주 인도 등 8개국의 관련 협회가 소속돼 있다. 이창헌 한국M&A투자협회 회장은 지난 5일까지 초대 회장을 맡았다.포럼 참석자들에 따르면 중국의 정·재계는 최근 한국 등 아시아 기업 M&A에 눈을 돌리고 있다. 동북아 실크로드 지역 개발과 경제 협력을 핵심으로 하는 일대일로 정책의 기업판이다. 양슝 상하이 시장은 “M&A를 통해 공급과잉 산업이 재편되고 성장률이 둔화되기 시작한 중국 경제에 활력을 불어넣을 것”이라고 말했다.민간차원에서 M&A를 활성화하기 위한 노력도 이어지고 있다. 중국M&A노조는 최근 상하이 시내에 M&A박물관을 개설하고 관련 교육시설로 활용하고 있다. 한편 6일 한국M&A투자협회 주최로 선전 증권거래소에서 열린 ‘한·중투자포럼’에서는 중국 칭하이하이런M&A펀드 등 수십 개의 기관 관계자들이 참석했다. 이날 국내 의료기기업체 하이로닉과 하이패스 제조업체 이씨스, 전자기기업체 씨엠파트너 등이 이들을 상대로 투자설명회를 열었다. 이창헌 회장은 “중국 기업들의 한국 투자 확대를 유도해 더 많은 국내 기업이 중국에 진출할 수 있도록 지원해나갈 것”이라고 말했다. 상하이·선전=정소람 기자'],
      dtype=object)

In [ ]:
def add_ir_context(df):
  irrelevant_contexts = []
  for idx, row in df.iterrows():
    title = row['title']
    irrelevant_contexts.append(
        df.query(f"title != '{title}'").sample(1)['context'].values[0]
    )

  df['irrelevant_context'] = irrelevant_contexts
  return df

df_train_ir = add_ir_context(df_train)
df_test_ir = add_ir_context(df_test)

In [ ]:
from sentence_transformers import InputExample

examples = []
for idx, row in df_train_ir.iterrows():
  examples.append(
      InputExample(texts=[row['question'], row['context']], label=1)
  )
  examples.append(
      InputExample(texts=[row['question'], row['irrelevant_context']], label=0)
  )

In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

evaluator = EmbeddingSimilarityEvaluator.from_input_examples(examples)
evaluator(sentence_model)

{'pearson_cosine': 0.8025437627931571, 'spearman_cosine': 0.8110220076830427}

## 2. Fine-Tuning

In [ ]:
train_samples = []
for idx, row in df_train_ir.iterrows():
  train_samples.append(
      InputExample(texts=[row['question'], row['context']])
  )

In [ ]:
from sentence_transformers import datasets

batch_size = 16

loader = datasets.NoDuplicatesDataLoader(
    train_samples, batch_size=batch_size
)

In [ ]:
from sentence_transformers import losses

loss = losses.MultipleNegativesRankingLoss(sentence_model)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
epochs = 1
save_path = './klue_mrc_mnr'

sentence_model.fit(
    train_objectives=[(loader, loss)],
    epochs=epochs,
    warmup_steps=len(loader) * 5,
    output_path=save_path,
    show_progress_bar=True
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.265500
1000,0.152500


In [ ]:
evaluator(sentence_model)

{'pearson_cosine': 0.912370189244613, 'spearman_cosine': 0.8606775188225755}

In [ ]:
from huggingface_hub import login, HfApi

login(token='hf_...')
repo_id = 'whatwant/klue-roberta-base-klue-sts-mrc'

api = HfApi()
api.create_repo(repo_id=repo_id)

api.upload_folder(
    folder_path=save_path,
    repo_id=repo_id,
    repo_type='model'
)

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/whatwant/klue-roberta-base-klue-sts-mrc/commit/b99cdb04bc4dce3f280a2494e1d6927fc24b88c8', commit_message='Upload folder using huggingface_hub', commit_description='', oid='b99cdb04bc4dce3f280a2494e1d6927fc24b88c8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/whatwant/klue-roberta-base-klue-sts-mrc', endpoint='https://huggingface.co', repo_type='model', repo_id='whatwant/klue-roberta-base-klue-sts-mrc'), pr_revision=None, pr_num=None)